## Classification MLPs

In [3]:
import tensorflow as tf

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

In [8]:
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

In [9]:
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [10]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]


In [11]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

2023-02-19 12:13:33.955705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Building Complex Models Using the Functional API

In [23]:
X_train.shape

(11610, 8)

In [76]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

# set up network
normalization_layer = layers.Normalization()
hidden_layer1 = layers.Dense(30, activation='relu')
hidden_layer2 = layers.Dense(30, activation='relu')
concat_layer = layers.Concatenate()
out_layer = layers.Dense(1)

input_ = layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)  # (N, 8)
hidden1 = hidden_layer1(normalized)  # (N, 30)
hidden2 = hidden_layer2(hidden1)  # (N, 30)
concat = concat_layer([normalized, hidden2])  # (N, 38)
output = out_layer(concat)  #(N, 1)

In [77]:
input_.shape

TensorShape([None, 8])

In [78]:
normalized.shape

TensorShape([None, 8])

In [79]:
hidden1.shape

TensorShape([None, 30])

In [80]:
hidden2.shape

TensorShape([None, 30])

In [81]:
concat.shape

TensorShape([None, 38])

In [82]:
output.shape

TensorShape([None, 1])

### Sending Stuff Through Different Paths

In [85]:
input_wide = tf.keras.layers.Input(shape=[5], name='input_wide')  # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6], name='input_deep')  # features 2 to 7
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), 
              optimizer=optimizer, metrics=["RootMeanSquaredError"])

X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)
x_dict_train = {'input_wide': X_train_wide, 'input_deep': X_train_deep}
x_dict_test = {'input_wide': X_test_wide, 'input_deep': X_test_deep}

In [85]:
history = model.fit(x_dict_train, y_train, epochs=20,
                    validation_data=((X_valid_wide, X_valid_deep), y_valid))

In [89]:
eval_results = model.evaluate(x_dict_test, y_test)
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 [==============================] - 0s 1ms/step - loss: 0.3400 - dense_92_loss: 0.3335 - dense_93_loss: 0.3989 - dense_92_root_mean_squared_error: 0.5775 - dense_93_root_mean_squared_error: 0.6316


In [96]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 [==============================] - 0s 22ms/step


In [97]:
y_pred

{'dense_92': array([[0.3802902],
        [1.2770959],
        [3.5700347]], dtype=float32),
 'dense_93': array([[0.50115055],
        [1.2095529 ],
        [3.3265321 ]], dtype=float32)}

## Subclassing API